In [1]:
from utils import get_path, group_cols
from datatable import dt, f

# Descrição

Neste notebook iremos formar uma tabela contendo todos os municípios de residência que estão na base do SIH.

# Tabelas base

## SIH

### Colunas da tabela

In [2]:
cols_mun = [
  'cod_municipio',
  'regiao',
  'uf',
  'cod_uf',
  'nome_uf',
  'municipio',
  'nome_municipio',
  'capital',
  'fronteira',
  'amazonia',
  'macroregiao_saude',
  'regiao_saude',
  'nome_regiao_saude',
  'microregiao_saude',
  'latitude',
  'longitude',
  'altitude',
  'area',
]

In [3]:
path_sih = get_path('SIH', 'SIH.jay')
df_mun = dt.fread(path_sih)
df_mun = df_mun[:, cols_mun]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,microregiao_saude,latitude,longitude,altitude,area
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,120005,Norte,AC,12,ACRE,Assis Brasil,ASSIS BRASIL,N,S,S,…,12900,−10.941,−69.567,239,2875.91
1,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
2,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
3,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
4,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
5,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
6,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58
7,120005,Norte,AC,12,ACRE,Assis Brasil,ASSIS BRASIL,N,S,S,…,12900,−10.941,−69.567,239,2875.91
8,120040,Norte,AC,12,ACRE,Rio Branco,RIO BRANCO,S,S,S,…,12900,−9.975,−67.81,153,9222.58


### Tabela agrupada

In [4]:
df_mun = group_cols(df_mun, cols_mun)
df_mun = df_mun.sort(-f.count)
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,latitude,longitude,altitude,area,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,−23.548,−46.636,760,1522.99,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,−22.903,−43.208,2,1182.3,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,−3.102,−60.025,92,11401.1,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,−3.717,−38.543,21,313.14,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,−12.971,−38.511,8,706.799,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,−1.456,−48.504,10,1064.92,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,−19.817,−43.956,858,330.954,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,−30.033,−51.23,3,496.827,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,−25.428,−49.273,934,434.967,101337


## Índice Socioeconômico

### Colunas da tabela

In [5]:
cols_socioecon = {
  'Cód IBGE': 'cod_municipio',
  'Grupo': 'socioecon',
  ...: None
}

In [6]:
path_socioeco = get_path('HEALTH', 'health_regions.csv')
df_socioeco = dt.fread(path_socioeco, columns=cols_socioecon)
df_socioeco

,cod_municipio,socioecon
,▪▪▪▪,▪▪▪▪
0,120001,2
1,120005,1
2,120010,1
3,120013,2
4,120017,2
5,120020,1
6,120025,1
7,120030,1
8,120032,2


# Gerando tabela

## Juntando com outras

## Juntando com Socioeconômico

In [7]:
df_socioeco.key = 'cod_municipio'
df_mun = df_mun[:, :, dt.join(df_socioeco)]
cols_mun = list(df_mun.names)
cols_mun = cols_mun[:-2] + [cols_mun[-1]] + [cols_mun[-2]]
df_mun = df_mun[:, cols_mun]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,longitude,altitude,area,socioecon,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,S,N,N,…,−46.636,760,1522.99,5,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,S,N,N,…,−43.208,2,1182.3,5,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,S,N,S,…,−60.025,92,11401.1,4,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,S,N,N,…,−38.543,21,313.14,5,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,S,N,N,…,−38.511,8,706.799,5,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,S,N,S,…,−48.504,10,1064.92,4,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,S,N,N,…,−43.956,858,330.954,1,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,S,N,N,…,−51.23,3,496.827,5,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,S,N,N,…,−49.273,934,434.967,5,101337


## Ajustando valores

### Valor Booleanos

In [14]:
df_mun[:, dt.update(capital = f.capital == 'S')]
df_mun[:, dt.update(fronteira = f.fronteira == 'S')]
df_mun[:, dt.update(amazonia = f.amazonia == 'S')]
df_mun

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,longitude,altitude,area,socioecon,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪,▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,1,0,0,…,−46.636,760,1522.99,5,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,1,0,0,…,−43.208,2,1182.3,5,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,1,0,1,…,−60.025,92,11401.1,4,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,1,0,0,…,−38.543,21,313.14,5,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,1,0,0,…,−38.511,8,706.799,5,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,1,0,1,…,−48.504,10,1064.92,4,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,1,0,0,…,−43.956,858,330.954,1,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,1,0,0,…,−51.23,3,496.827,5,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,1,0,0,…,−49.273,934,434.967,5,101337


# Exportando tabela

In [15]:
output_mun = get_path('GESTANTES', 'municipios.csv.gzip')
df_mun.to_csv(output_mun)

In [16]:
dt.fread(output_mun)

,cod_municipio,regiao,uf,cod_uf,nome_uf,municipio,nome_municipio,capital,fronteira,amazonia,…,longitude,altitude,area,socioecon,count
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪,▪,▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪▪▪▪
0,355030,Sudeste,SP,35,SAO PAULO,São Paulo,SAO PAULO,1,0,0,…,−46.636,760,1522.99,5,806352
1,330455,Sudeste,RJ,33,RIO DE JANEIRO,Rio de Janeiro,RIO DE JANEIRO,1,0,0,…,−43.208,2,1182.3,5,368216
2,130260,Norte,AM,13,AMAZONAS,Manaus,MANAUS,1,0,1,…,−60.025,92,11401.1,4,297036
3,230440,Nordeste,CE,23,CEARA,Fortaleza,FORTALEZA,1,0,0,…,−38.543,21,313.14,5,203370
4,292740,Nordeste,BA,29,BAHIA,Salvador,SALVADOR,1,0,0,…,−38.511,8,706.799,5,169924
5,150140,Norte,PA,15,PARA,Belém,BELEM,1,0,1,…,−48.504,10,1064.92,4,126918
6,310620,Sudeste,MG,31,MINAS GERAIS,Belo Horizonte,BELO HORIZONTE,1,0,0,…,−43.956,858,330.954,1,125370
7,431490,Sul,RS,43,RIO GRANDE DO SUL,Porto Alegre,PORTO ALEGRE,1,0,0,…,−51.23,3,496.827,5,105321
8,410690,Sul,PR,41,PARANA,Curitiba,CURITIBA,1,0,0,…,−49.273,934,434.967,5,101337
